In [124]:
from time import time
import numpy as np
import string
from tqdm.autonotebook import tqdm
from techniques import *
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
lemmatizer = WordNetLemmatizer() # set lemmatizer
stemmer = PorterStemmer() # set stemmer

In [125]:
### Glove related. Function to load a glove file. You need to download it. 
def loadGloveModel(gloveFile):
    '''
    Load a Glove file. Please download it from http://nlp.stanford.edu/data/glove.twitter.27B.zip
    '''
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf-8')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

# Load the model. Change the path if needed
model = loadGloveModel('glove.twitter.27B.200d.txt')

Loading Glove Model
Done. 1193514  words loaded!


### We need to generate an average of Glove to represent OOV (out-of-vocabulary) words
#### The actual codes are commented out to save time. 

In [8]:
# ## Generate average vec start
# GLOVE_FILE = 'glove.twitter.27B.200d.txt'
# # Get number of vectors and hidden dim
# with open(GLOVE_FILE, 'r',encoding="utf8") as f:
#     for i, line in tqdm(enumerate(f)):
#         pass
# n_vec = i + 1
# hidden_dim = len(line.split(' ')) - 1

# vecs = np.zeros((n_vec, hidden_dim), dtype=np.float32)

# with open(GLOVE_FILE, 'r',encoding="utf8") as f:
#     for i, line in tqdm(enumerate(f)):
#         vecs[i] = np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)

# oov = np.mean(vecs, axis=0)
# ## Generate average vec end

oov = np.array([-0.0940983 , -0.02843253, -0.01981537,  0.00476221,  0.00562878,
       -0.08310315, -0.10455726, -0.11054859,  0.04218907, -0.05685968,
       -0.07577542,  0.01502523,  0.12450685, -0.03634054,  0.0405103 ,
        0.01487466,  0.03977518,  0.04460221,  0.09106229,  0.05568037,
        0.02800928,  0.06613768, -0.01108837,  0.01697896, -0.10412963,
        0.3507236 ,  0.02584417,  0.00360701, -0.18075785,  0.00826703,
        0.15338054,  0.09718955,  0.18591996, -0.12195268, -0.19989698,
       -0.12006074,  0.0120836 ,  0.01818978,  0.10259497,  0.02875553,
        0.39889172, -0.00370644,  0.03968571, -0.041596  , -0.13434792,
        0.10075227, -0.05748982,  0.00397006,  0.08035477,  0.12797299,
        0.08266383,  0.05858067, -0.04357374, -0.08068992, -0.04668539,
       -0.06677692,  0.04721875,  0.00727302, -0.05912784, -0.04276676,
        0.09697088,  0.04666249,  0.02529906,  0.04875451, -0.17676166,
        0.19386147,  0.12038179,  0.08816861, -0.0571619 , -0.00090087,
       -0.11305717,  0.05075013, -0.05707799, -0.03146713,  0.03141678,
        0.14196746,  0.09224427, -0.01728742,  0.01046179,  0.03585972,
       -0.09386539, -0.09762925, -0.06350451, -0.00260555, -0.0661344 ,
       -0.0807054 ,  0.1642215 , -0.2894754 ,  0.00306663, -0.05767346,
        0.00879237,  0.12618215,  0.05366566,  0.07843259,  0.03640802,
       -0.00649674,  0.01998443,  0.06910556, -0.13995245,  0.04371282,
       -0.06699623, -0.00429413, -0.09469022, -0.09512661, -0.12079419,
       -0.1227353 ,  0.11687743, -0.01618212,  0.13010803, -0.07520641,
        0.08611171, -0.11750808, -0.03315089,  0.03394032, -0.04863394,
       -0.04742027,  0.03740013,  0.05302504,  0.0292228 ,  0.00939865,
       -0.02425062,  0.00254103,  0.04668314, -0.02513516,  0.01587607,
        0.13379093, -0.02098919, -0.02138812,  0.12004584, -0.12496009,
        0.09164985,  0.02535828,  0.030993  ,  0.01908904, -0.1135054 ,
       -0.05747874, -0.00097564, -0.07191793, -0.04650274,  0.09658901,
       -0.03481024,  0.07673627,  0.07474707, -0.12794939, -0.03416433,
       -0.125318  ,  0.05369347, -0.08551957,  0.00373768,  0.01549865,
        0.11278545, -0.0193907 ,  0.8663707 , -0.09132162, -0.04578998,
        0.08276154, -0.05723441, -0.04189969,  0.04966758,  0.0884276 ,
        0.07552473, -0.08842846, -0.03441407,  0.02968801,  0.03326621,
       -0.01641093,  0.02045972, -0.04570299, -0.04881294,  0.03214714,
        0.08044318,  0.05654246, -0.0202872 , -0.06148338,  0.029538  ,
       -0.12749897, -0.09602174, -0.09560092,  0.08549695, -0.081919  ,
       -0.06052832,  0.01137542, -0.01080422, -0.08540731,  0.00334476,
        0.18626893,  0.13344373,  0.0643768 , -0.06667358,  0.04245436,
        0.10244108,  0.00752079,  0.06437495,  0.05950141,  0.11263546,
       -0.01919787,  0.255039  ,  0.14020036,  0.08237398, -0.2602364 ])


In [138]:
### Codes are from https://github.com/Deffro/text-preprocessing-techniques

""" Tokenizes a text to its words, removes and replaces some of them """    
finalTokens = [] # all tokens
stoplist = stopwords.words('english')
my_stopwords = "url atuser st rd nd th am pm atus" # my extra stopwords
stoplist = stoplist + my_stopwords.split()
allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging

def tokenize(text, wordCountBefore, textID, y, file):
    totalAdjectives = 0
    totalAdverbs = 0
    totalVerbs = 0
    onlyOneSentenceTokens = [] # tokens of one sentence each time

    tokens = nltk.word_tokenize(text)
    
    tokens = replaceNegations(tokens) # Technique 6: finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym
    
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator) # Technique 7: remove punctuation

    tokens = nltk.word_tokenize(text) # it takes a text as an input and provides a list of every token in it

#     ### POS TAGGING BEGIN (If you want to exclude words using POS Tagging, keep this section uncommented and comment the above) ###          
    tagged = nltk.pos_tag(tokens) # Technique 13: part of speech tagging  
    for w in tagged:

        if (w[1][0] in allowedWordTypes and w[0] not in stoplist):
            final_word = addCapTag(w[0])
            #final_word = final_word.lower()
            final_word = replaceElongated(final_word)
            if len(final_word)>1:
                final_word = spellCorrection(final_word)
#             final_word = lemmatizer.lemmatize(final_word)
#             final_word = stemmer.stem(final_word)

#     # ### POS TAGGING END ###
## NO POS TAGGING BEGIN (If you don't want to use POS Tagging keep this section uncommented) ###

#     for w in tokens:
#         if (w not in stoplist): # Technique 10: remove stopwords
#             final_word = addCapTag(w) # Technique 8: Finds a word with at least 3 characters capitalized and adds the tag ALL_CAPS_
#             final_word = final_word.lower() # Technique 9: lowercases all characters
#             final_word = replaceElongated(final_word) # Technique 11: replaces an elongated word with its basic form, unless the word exists in the lexicon
#             if len(final_word)>1:
#                 final_word = spellCorrection(final_word) # Technique 12: correction of spelling errors
#             final_word = lemmatizer.lemmatize(final_word) # Technique 14: lemmatizes words
#             final_word = stemmer.stem(final_word) # Technique 15: apply stemming to words

## NO POS TAGGING END ###
                
            onlyOneSentenceTokens.append(final_word)           
            finalTokens.append(final_word)

         
    onlyOneSentence = " ".join(onlyOneSentenceTokens) # form again the sentence from the list of tokens
    #print(onlyOneSentence) # print final sentence

    
    """ Write the preprocessed text to file """
    with open(file, "a") as result:
        result.write(textID+"\t"+y+"\t"+onlyOneSentence+"\n")
        
    return finalTokens

    

### Block of codes to tokenize text data. The result does not have the features, only tokens. You can ignore it since I have also included the final output in train.csv and test.csv. 
### So the input for this is raw data, let's name the output A. 

In [127]:
# f1 = open("twitter-2016train-A.txt","r", encoding="utf8", errors='replace').read()

# t0 = time()
# totalSentences = 0
# totalEmoticons = 0
# totalSlangs = 0
# totalSlangsFound = []
# totalElongated = 0
# totalMultiExclamationMarks = 0
# totalMultiQuestionMarks = 0
# totalMultiStopMarks = 0
# totalAllCaps = 0

# for line in tqdm(f1.split('\n')):
#     totalSentences += 1
#     feat = []
#     columns = line.split('\t')
#     columns = [col.strip() for col in columns]

#     textID = (columns[0])
#     y = (columns[1])

#     text = removeUnicode(columns[2]) # Technique 0
#     #print(text) # print initial text
#     wordCountBefore = len(re.findall(r'\w+', text)) # word count of one sentence before preprocess    
#     #print("Words before preprocess: ",wordCountBefore,"\n")
    
#     text = replaceURL(text) # Technique 1
#     text = replaceAtUser(text) # Technique 1
#     text = removeHashtagInFrontOfWord(text) # Technique 1

#     temp_slangs, temp_slangsFound = countSlang(text)
# #     totalSlangs += temp_slangs # total slangs for all sentences
#     for word in temp_slangsFound:
#         totalSlangsFound.append(word) # all the slangs found in all sentences
    
#     text = replaceSlang(text) # Technique 2: replaces slang words and abbreviations with their equivalents
#     text = replaceContraction(text) # Technique 3: replaces contractions to their equivalents
#     text = removeNumbers(text) # Technique 4: remove integers from text

# #     emoticons = countEmoticons(text) # how many emoticons in this sentence
# #     totalEmoticons += emoticons
    
#     text = removeEmoticons(text) # removes emoticons from text

    
# #     totalAllCaps += countAllCaps(text)

#     totalMultiExclamationMarks += countMultiExclamationMarks(text) # how many repetitions of exlamation marks in this sentence
#     totalMultiQuestionMarks += countMultiQuestionMarks(text) # how many repetitions of question marks in this sentence
#     totalMultiStopMarks += countMultiStopMarks(text) # how many repetitions of stop marks in this sentence
    
#     text = replaceMultiExclamationMark(text) # Technique 5: replaces repetitions of exlamation
#     text = replaceMultiQuestionMark(text) # Technique 5: replaces repetitions of question marks 
#     text = replaceMultiStopMark(text) # Technique 5: replaces repetitions of stop marks 
#     totalElongated += countElongated(text) # how many elongated words emoticons in this sentence
    
#     tokens = tokenize(text, wordCountBefore, textID, y, 'result-2016train-A.txt')  

# f2 = open("twitter-2016test-A.txt","r", encoding="utf8", errors='replace').read()
 
# for line in tqdm(f2.split('\n')):
#     totalSentences += 1
#     feat = []
#     columns = line.split('\t')
#     columns = [col.strip() for col in columns]

#     textID = (columns[0])
#     y = (columns[1])

#     text = removeUnicode(columns[2]) # Technique 0
#     #print(text) # print initial text
#     wordCountBefore = len(re.findall(r'\w+', text)) # word count of one sentence before preprocess    
#     #print("Words before preprocess: ",wordCountBefore,"\n")
    
#     text = replaceURL(text) # Technique 1
#     text = replaceAtUser(text) # Technique 1
#     text = removeHashtagInFrontOfWord(text) # Technique 1

#     temp_slangs, temp_slangsFound = countSlang(text)
# #     totalSlangs += temp_slangs # total slangs for all sentences
#     for word in temp_slangsFound:
#         totalSlangsFound.append(word) # all the slangs found in all sentences
    
#     text = replaceSlang(text) # Technique 2: replaces slang words and abbreviations with their equivalents
#     text = replaceContraction(text) # Technique 3: replaces contractions to their equivalents
#     text = removeNumbers(text) # Technique 4: remove integers from text

# #     emoticons = countEmoticons(text) # how many emoticons in this sentence
# #     totalEmoticons += emoticons
    
#     text = removeEmoticons(text) # removes emoticons from text

    
# #     totalAllCaps += countAllCaps(text)

#     totalMultiExclamationMarks += countMultiExclamationMarks(text) # how many repetitions of exlamation marks in this sentence
#     totalMultiQuestionMarks += countMultiQuestionMarks(text) # how many repetitions of question marks in this sentence
#     totalMultiStopMarks += countMultiStopMarks(text) # how many repetitions of stop marks in this sentence
    
#     text = replaceMultiExclamationMark(text) # Technique 5: replaces repetitions of exlamation
#     text = replaceMultiQuestionMark(text) # Technique 5: replaces repetitions of question marks 
#     text = replaceMultiStopMark(text) # Technique 5: replaces repetitions of stop marks 
#     totalElongated += countElongated(text) # how many elongated words emoticons in this sentence
    
#     tokens = tokenize(text, wordCountBefore, textID, y, 'result-2016test-A.txt')  

In [107]:
files = ['result-2016test-A', 'result-2016train-A']

### Block of codes to generate features.  You can ignore it since I have also included the final output in train.csv and test.csv
### The input of this block is A, let's name the output B.

In [108]:
# for file in files:
#     data = pd.read_csv(file+'.txt', sep='\t',header = None, encoding='latin-1')
#     data.columns = ['id', 'label', 'tokens']
#     data.loc[data['label'] == 'positive', 'label']=1
#     data.loc[data['label'] == 'neutral', 'label']=0
#     data.loc[data['label'] == 'negative', 'label']=-1
#     for i, row in tqdm(data.iterrows()):
#         token = row['tokens'].split(' ')
#         feature = []
#         for word in token :
#             # Use oov if word is not in model
#             if (word not in model):
#                 feature.append( oov)
#             else:
#                 feature.append(model[word])
#         series = pd.Series(np.average(feature, axis = 0))
#         for index, x in series.items():
#             data.loc[i,'feat_'+str(index)] = x 
#     data.to_csv(file + '.csv', encoding='latin-1', index = False)

### Block of codes to process data generated from the tokens.  You can ignore it since I have also included the final output in train.csv and test.csv
### The input of this block is B, let's name the output C, which are train.csv and test.csv

In [128]:
# # Read in data into panda dataframe. 
# test = pd.read_csv('result-2016test-A.txt', sep='\t',header = None,  encoding='latin-1')
# test.columns = ['id', 'label', 'tokens']
# # Read in data into panda dataframe. 
# train = pd.read_csv('result-2016train-A.txt', sep='\t',header = None,  encoding='latin-1')
# train.columns = ['id', 'label', 'tokens']
# # Change the labels from String to Integer
# test.loc[test['label'] == 'positive', 'label']=1
# test.loc[test['label'] == 'neutral', 'label']=0
# test.loc[test['label'] == 'negative', 'label']=-1
# # Change the labels from String to Integer
# train.loc[train['label'] == 'positive', 'label']=1
# train.loc[train['label'] == 'neutral', 'label']=0
# train.loc[train['label'] == 'negative', 'label']=-1

In [154]:
train = pd.read_csv('result-2016train-A.csv',  encoding='latin-1')
test = pd.read_csv('result-2016test-A.csv',  encoding='latin-1')

In [155]:
y_train = train['label']

In [156]:
y_test = test['label']

In [157]:
X_train = train[train.columns[3:]]

In [158]:
X_test = test[train.columns[3:]]

In [159]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)

F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
F:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [160]:
clf.score(X_train, y_train)

0.6295

In [161]:
clf.score(X_test, y_test)

0.5046774271727013

In [162]:
from sklearn.neural_network import MLPClassifier

In [163]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(10,), random_state=1)

In [164]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [165]:
clf.score(X_train, y_train)

0.6868333333333333

In [153]:
clf.score(X_test, y_test)

0.46774271727012745